<a href="https://colab.research.google.com/github/its-parmiss/text-summarization/blob/main/Text_summariztion_with_LLMs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# Install necessary libraries
!pip install transformers datasets gradio
!pip install rouge_score --quiet

# Import libraries
from transformers import pipeline
import gradio as gr


  Preparing metadata (setup.py) ... done


In [5]:
!pip install transformers datasets evaluate --quiet

from transformers import pipeline
from datasets import load_dataset
import evaluate
from tqdm import tqdm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 2.0 MB/s eta 0:00:00


In [2]:
# Load the summarization pipeline
summarizer = pipeline("summarization", model="t5-small")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Device set to use cpu


In [3]:
def summarize_text(text):
    summary = summarizer(text, max_length=100, min_length=30, do_sample=False)
    return summary[0]['summary_text']


In [4]:
# Create Gradio interface
interface = gr.Interface(fn=summarize_text,
                         inputs="text",
                         outputs="text",
                         title="Text Summarizer",
                         description="Enter text to generate its summary.")

# Launch the interface
interface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://989083c5f5ee7c5ca4.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [6]:
# Load a small portion of CNN/DailyMail for demo
dataset = load_dataset("cnn_dailymail", "3.0.0", split="test[:5]")
dataset = dataset.select_columns(["article", "highlights"])  # Keep relevant columns


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [7]:
def summarize(text, max_len=120, min_len=30):
    result = summarizer(text, max_length=max_len, min_length=min_len, do_sample=False)
    return result[0]['summary_text']


In [8]:
preds = []
refs = []

for item in tqdm(dataset):
    summary = summarize(item['article'])
    preds.append(summary)
    refs.append(item['highlights'])


100%|██████████| 5/5 [00:30<00:00,  6.03s/it]


In [12]:
rouge = evaluate.load("rouge")
scores = rouge.compute(predictions=preds, references=refs)

print("ROUGE Evaluation Results:")
for k, v in scores.items():
    print(f"{k}: {v:.4f}")


ROUGE Evaluation Results:
rouge1: 0.3945
rouge2: 0.1280
rougeL: 0.2575
rougeLsum: 0.3307


In [13]:
def test_custom_summary():
    text = input("Paste a long article or text:\n")
    print("\n--- Summary ---\n")
    print(summarize(text))

# Run this cell and paste text when prompted
# test_custom_summary()


In [10]:
import gradio as gr

# Reuse the summarize function defined earlier

demo = gr.Interface(
    fn=summarize,
    inputs=gr.Textbox(lines=10, placeholder="Paste a news article or any long text..."),
    outputs="text",
    title="LLM Article Summarizer",
    description="Uses T5-small to summarize long text. Great for news, reports, or any document."
)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://58c3a01b0a4874cb8a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [15]:
!pip install git+https://github.com/openai/whisper.git --quiet
!pip install gradio --quiet
!apt-get install ffmpeg -y


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 914.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 52.1 MB/s et

In [16]:
import whisper
import gradio as gr

# Load the Whisper model
model = whisper.load_model("base")


100%|███████████████████████████████████████| 139M/139M [00:01<00:00, 86.0MiB/s]


In [17]:
def transcribe(audio_path):
    result = model.transcribe(audio_path)
    return result["text"]
